In [ ]:
!git clone https://github.com/SIH-2024-TEAM-VANGUARD/VANGUARD.git

Cloning into 'VANGUARD'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 21 (delta 4), reused 21 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 5.99 MiB | 8.45 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
%cd VANGUARD
!ls

/content/VANGUARD
context.txt  merged.pdf  README.md  requirements.txt  server.py  static  templates  test_index


In [ ]:
!pip install flask transformers torch
!pip install flask_cors
!pip install pyngrok
!pip install sentence_transformers
!pip install langchain chromadb pypdf accelerate
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.2 MB/s eta 0:

In [ ]:
from flask import Flask, render_template, url_for, request, jsonify
import pathlib
from pathlib import Path
import os
import json
import requests
from flask_cors import CORS
import torch
import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BertTokenizer, BertModel
tokenizer_emb = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_emb = BertModel.from_pretrained("bert-base-multilingual-cased")
tokenizer_slm = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
model_slm = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True,)
pipe = pipeline("text-generation", model=model_slm, tokenizer=tokenizer_slm, max_new_tokens=256)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)
#tokenizer_lg = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
#tokenizer_lg = tokenizer_slm
#model_lg = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True,)
#model_lg=model_slm
###pipe_lg = pipeline("text-generation", model=model_lg, tokenizer=tokenizer_lg, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

# Load the PDF file
#pdf_link = "https://cdnbbsr.s3waas.gov.in/s380537a945c7aaa788ccfcdf1b99b5d8f/uploads/2024/07/20240716890312078.pdf"
pdf_link = "//content//VANGUARD//merged.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()


# Split data into chunks
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size = 4000,
   chunk_overlap  = 20,
   length_function = len,
   add_start_index = True,
)
chunks = text_splitter.split_documents(pages)
# Store data into database
db=Chroma.from_documents(chunks,embedding=embeddings,persist_directory="test_index")
db.persist()
torch.cuda.empty_cache()
 #Load the database
vectordb = Chroma(persist_directory="test_index", embedding_function = embeddings)
 #Load the retriver
retriever = vectordb.as_retriever(search_kwargs = {"k" : 3})
# Define the custom prompt template suitable for the Phi-3 model
qna_prompt_template="""<|system|>
You have been provided with the context and a question, try to find out the answer to the question only using the context information. If the answer to the question is not found within the context, return "I dont know" as the response.<|end|>
<|user|>
Context:
{context}

Question: {question}<|end|>
<|assistant|>"""
PROMPT = PromptTemplate(
   template=qna_prompt_template, input_variables=["context", "question"]
)

#Define the QNA chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)
 # utility function for answer generation
def ask(question):
  prompt=""
  context = retriever.get_relevant_documents(question)
  answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
  return answer

#Setting functions for RAG
# Open the file in read mode
text=""""""
with open('//content//VANGUARD//context.txt', 'r') as file:
    # Read the entire content of the file
    text = file.read()

#model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = text.split('\n\n')
#embeddings = model.encode(sentences)
database=[]
vectorbase=[]
encoded_input = tokenizer_emb(sentences, return_tensors='pt', truncation=True, padding=True)


app = Flask(__name__)
CORS(app)


@app.route('/send_conversation', methods=['POST'])
def process_text():
    prompt=""""""
    data = request.get_json()
    torch.cuda.empty_cache()
    for individual in data['conversation']:
        prompt=prompt+individual['sender']+": "+individual['text']+"\n\n"
        try:
          prompt=prompt+individual['bot']+": "+individual['text']+"\n\n"
        except:
          print("no bot")

    # Process the text (for example, convert to uppercase)
    processed_text = ask(prompt)
    # Split the text by the delimiter
    answer = (processed_text.split("<|assistant|>")[-1]).strip()

    torch.cuda.empty_cache()
    return jsonify({'processed_text': answer})

@app.route('/send_conversation_summarize', methods=['POST'])
def process_text_summarize():
    prompt=""""""
    data = request.get_json()
    torch.cuda.empty_cache()
    for individual in data['conversation']:
        prompt=prompt+individual['sender']+": "+individual['text']+"\n\n"
        try:
          prompt=prompt+individual['bot']+": "+individual['text']+"\n\n"
        except:
          print("no bot")

    prompt="Summarize Accordingly:\n\n"+prompt
    # Process the text (for example, convert to uppercase)
    processed_text = ask(prompt)
    # Split the text by the delimiter
    answer = (processed_text.split("<|assistant|>")[-1]).strip()

    torch.cuda.empty_cache()
    return jsonify({'processed_text': answer})

@app.route('/send_conversation_getinfo', methods=['POST'])
def process_text_getinfo():
    prompt=""""""
    data = request.get_json()
    torch.cuda.empty_cache()
    for individual in data['conversation']:
        prompt=prompt+individual['sender']+": "+individual['text']+"\n\n"
        try:
          prompt=prompt+individual['bot']+": "+individual['text']+"\n\n"
        except:
          print("no bot")

    query=prompt
    encoded_query = tokenizer_emb(query, return_tensors='pt', truncation=True, padding=True)
    similarities=[]
    for i in encoded_input['input_ids']:
      j=[]
      j.append(i)
      similarities.append(np.sum(np.dot(encoded_query['input_ids'].T, j)))
    print(np.asarray(similarities).shape)
    top_3_idx=np.argsort(similarities)
    top_3_idx
    prompt=""
    for i in top_3_idx:
     prompt=prompt+sentences[i]

    prompt="From the given context\n\n "+prompt+"\n\nAnswer this query \n\n"+query
    # Generate text
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95, max_time=180)

    # Print the generated text
    answer = outputs[0]['generated_text'][len(prompt):].strip()
    torch.cuda.empty_cache()
    return jsonify({'processed_text': answer})

@app.route('/alive')
def beep():
    return jsonify("Hello, are you alive ?"), 200

@app.route("/")
def home():
    return render_template("index1.html")

@app.route('/summarize')
def summarize():
    return render_template('summarize.html')

@app.route('/getinfo')
def getinfo():
    return render_template('getinfo.html')



#if __name__ == '__main__':
#    app.run(debug=True, port=8080)



from pyngrok import ngrok

!ngrok authtoken 2UFfKWMWbMuAC3aMQ8NN9ree3ib_6EjAy5T5V3kg8fR2Ce6gN

# Set up the ngrok tunnel
public_url = ngrok.connect(3000)

print('Public URL:', public_url)
#if _name_ == '_main_':
#    app.run(host='0.0.0.0', port=3000)
#    #app.run(debug=True, port=3000)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-4-ab41ae7b0550>:35: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
<ipython-input-4-ab41ae7b0550>:54: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
<ipython-input-4-ab41ae7b0550>:57: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chrom

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://ae3f-34-125-158-47.ngrok-free.app" -> "http://localhost:3000"


In [ ]:
app.run(host='0.0.0.0', port=3000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3000
 * Running on http://172.28.0.2:3000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:06:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:06:34] "GET /static/css/styles1.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:06:34] "GET /static/js/chat1.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:06:40] "GET /favicon.ico HTTP/1.1" 404 -


no bot


<ipython-input-4-ab41ae7b0550>:78: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = retriever.get_relevant_documents(question)
<ipython-input-4-ab41ae7b0550>:79: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))['output_text']
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


no bot
no bot
no bot
no bot
no bot
no bot
no bot
no bot
no bot


...

ERROR:__main__:Exception on /send_conversation [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask_cors/extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-4-ab41ae7b0550>", line 114, in process_text
    processed_text = ask(prompt)
  File "<ipython-input-4-ab41ae7b0550>", line 79, in ask
    answer

no bot
no bot
no bot
no bot
no bot
no bot
no bot
no bot


INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:09:59] "POST /send_conversation HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:09:59] "POST /send_conversation HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:10:09] "POST /send_conversation HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:11:03] "GET /getinfo HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:11:03] "GET /static/css/styles1.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2024 08:11:03] "GET /static/js/chat1_getinfo.js HTTP/1.1" 200 -


no bot
(128,)


ERROR:__main__:Exception on /send_conversation_getinfo [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask_cors/extension.py", line 194, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-4-ab41ae7b0550>", line 170, in process_text_getinfo
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperat